In [ ]:
import pandas as pd
from datetime import datetime
import pywhatkit as wpp

Funciones

In [ ]:
def dias():
    reference_day = datetime(2024, 1, 8)
    today = datetime.now()
    deltaday = (today - reference_day).days

    sem = int(deltaday // 7 % 4)
    mes = today.month - reference_day.month +1
    year = 2024
    return sem, mes, year

In [ ]:
def clean_asig(df):
    # Solo Colunnas necesarias. Filtro solo Cadereyta
    
    df = df[['MATRICULA','NOMBRE', 'CE', 'MATERIA']]
    
    # Matriculas en MAYUS
    df['MATRICULA'] = df['MATRICULA'].str.upper()
    
    cade = ('CADEREYTA', 'CE CADEREYTA', 'CC CADEREYTA')
    df = df[df['CE'].str.contains('|'.join(cade), case = False, na = False)]
    return df

In [ ]:
def colab():
    print('Grupos Colaborativos')
    
    materias_dict = {'cal dif':['Cálculo diferencial', 'https://chat.whatsapp.com/K2y5qOtGQeiAcNOSD77dWv'],'admin':['Administración I',], 'integral':['Cálculo integral',], 'fun mat':['Fundamentos de matemáticas', 'https://chat.whatsapp.com/Env3dGXvLIl7HYXWFUtICE'], 'conta':['Introducción a la contabilidad', ],'derecho':['Introducción al derecho', ]}
    print(materias_dict.keys())
    
    grupo = str(input('Grupo ?'))
    link = materias_dict[grupo][1]
    grupo = materias_dict[grupo][0]
    try: 
        asig = pd.read_csv(path + "Asignacion.csv", index_col= 0)
    except FileNotFoundError or FileExistsError:
        mes = datetime.now().month
        asig = extras = pd.read_excel(path + main + "Román - M{} Seguimiento.xlsx".format(mes), sheet_name = "Asignación")
        clean_asig(asig).to_csv(path + 'Asignacion.csv')
        print('Asignacion csv Created')
        asig = pd.read_csv(path + "Asignacion.csv", index_col= 0)
    if grupo:
        asig = asig[asig['MATERIA'] == grupo]
    asig_merged = pd.merge(asig, database[['MATRICULA', 'Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return asig_merged, link

In [ ]:
def test():
    print('Enviar Test de Aprendizaje')
    path = 'Data/'
    df = pd.read_csv(path + 'PendientesTest.csv')
    df_merge = pd.merge(df, database[['MATRICULA', 'Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return df_merge

In [ ]:
def result_test():
    print('Resultados Test de Aprendizaje')
    path = 'Data/'
    df = pd.read_csv(path + 'resultest.csv')
    df_merge = pd.merge(df, database[['MATRICULA', 'Teléfono', 'Teléfono2']], on = 'MATRICULA')
    return df_merge

In [ ]:
def noinicios():
    print('No Inicios')
    path = 'Data/'
    main = 'Main/'
    try:
        df = pd.read_csv('noinicios.csv')
    except FileNotFoundError or FileExistsError:
        sem , mes, year = dias()
        df = pd.read_excel(path + main +"BASE BLENDED NL S{:d}M{:02d} {}.xlsx".format(sem, mes, year), sheet_name = 'NO INICIO')
        matriculas = pd.read_csv('infodata/matriculas_noinicios.csv', header=None)
        matriculas = matriculas.iloc[0].tolist()
        df = df[['TIPO DE BAJA', 'NOMBRE EN LA BASE', 'MATRICULA EDCON','CE CANALIZADO','Fecha','SERVICIO','CARRERA','M1','Fecha de inicio','Correo','Teléfono','Teléfono 2','Asesor Financiero','Asesor Escolar',]]
        df = df.rename(columns={'NOMBRE EN LA BASE':'NOMBRE','MATRICULA EDCON':'MATRICULA', 'Teléfono 2':'Teléfono2'})
        df = df[df['MATRICULA'].isin(matriculas)]
    return df

Codigo

In [ ]:
path = 'Data/'
main = 'Main/'

try:
    database = pd.read_csv(path + "BASE CNCI.csv") 
except FileNotFoundError or FileExistsError: 
    %run DatabaseCNCI.ipynb
    print('Database CNCI.csv Created')
    database = pd.read_csv(path + "BASE CNCI.csv")

In [ ]:
path2 = 'infodata/'
imagen = 'img/Cerro.jpeg'

#Inicializar la infodata
matric_error = pd.read_csv('infodata/matriculas_error.csv')
matric_error = matric_error.dropna().values.flatten().tolist()
    
try:    
    matric_yacontactadas = pd.read_csv(path2+ 'matriculas_yacontactadas.csv') 
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist() 
except FileNotFoundError:
    matric_yacontactadas = pd.DataFrame()

Opciones

In [ ]:
option = 0
while option not in (1, 2, 3):
    option = int(input('1.- Colab \n2.- Test Aprendizaje \n3.- No inicios'))
    if option == 1: 
        df, link = colab()
    elif option == 2:
        option2 = int(input('1.- Envio \n2.- Resultados '))
        if option2 == 1:
            link = 'https://forms.office.com/r/xjAa2rPUhf'
            df = test()
        elif option2 == 2:
            df = result_test()
    elif option == 3:
        df = noinicios()

Ciclo de mensajes

In [ ]:
for index, row in df.iterrows():
    alumno = row['NOMBRE']
    matricula = row['MATRICULA']
    telefono = row['Teléfono']
    
    if option == 1:
        materia = row['MATERIA']
        mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ tu Tutor Presencial del CNCI, me comunico contigo para invitarte al grupo colaborativo de la materia: *'+ materia + '* con la finalidad de que podamos entablar conversación por ahí sobre los temas relacionados y además tener apoyo entre los compañeros. \n\nSi gustas a continuación te dejo el enlace: ' + link
    elif option == 2:
        if option2 == 1:
            mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ tu Tutor Presencial del CNCI, me comunico contigo para invitarte a contestar el *Test de Aprendizaje*, el cual nos ayuda a saber cuál es la mejor manera en la que cada uno se le facilita aprender.\n Una vez contestado te agradecería que me mandes captura de que ya lo contestaste. \n\n Te dejo el enlace: '+link
        elif option2 == 2:
            resultado = row['ESTILO'].title() + ' ' + row['Resultado'].title()
            mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', te saludo de vuelta _Román Rodríguez_ Tutor Presencial del CNCI, me comunico contigo para enviarte los resultados del _Test de Aprendizaje_ que contestaste, con el cual obtuvimos el *mejor método de aprendizaje* para tí. Tu _Estilo de Aprendizaje_ es el siguiente: *'+ resultado +'*'
            imagen = 'img/ResultadosTestAprendizaje/'+ resultado+'.png'
    elif option == 3:
        mensaje = 'Hola, que tal ' + alumno + '. Me presento soy _Román Rodríguez_ Tutor Presencial del CNCI, me comunico contigo para invitarte a *RETOMAR* tu curso de *Inducción*, así dar un nuevo inicio a tu carrera, el próximo inicio es el 13 de mayo, mencioname si estas interesad@ y con gusto te contacto, ten un buen día!'

    if not pd.isna(telefono):
        telefono = '+52 ' + str(telefono)
    if matricula in matric_error:
        if not pd.isna(row['Teléfono2']):
            telefono2 = '+52 ' + str(row['Teléfono2'])
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))
                 
    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, Alumno: {} '.format(matricula, alumno))    
    else:
        print('Enviado a: {} {}'.format(matricula, alumno), telefono)
        print('--------------------')
        # wpp.sendwhats_image(telefono, imagen, mensaje, 25, True)

Prototipo de chequeo de imagenes

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Lee la imagen
# print(alumno, resultado)
figure = Image.open(imagen)
# Muestra la imagen
plt.imshow(figure)
plt.axis('off')  # Desactiva los ejes
plt.show()